In [46]:
import csv
from datetime import datetime
from datetime import timedelta
import json
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import requests
import time
from turtle import title
from urllib.request import urlopen

ETH = '0x82aF49447D8a07e3bd95BD0d56f35241523fBab1'
WBTC = '0x2f2a2543B76A4166549F7aaB2e75Bef0aefC5B0f'
LINK = '0xf97f4df75117a78c1A5a0DBb814Af92458539FB4'
UNI = '0xFa7F8980b0f1E64A2062791cc3b0871572f1F7f0'
# today_date = datetime.now().strftime("%m/%d/%Y")

#change to include other coins
coins = {ETH:'ETH/USD',WBTC:'WBTC/USD'}
wbtc_data = []
eth_data = []

prices_url = "https://api.gmx.io/prices" #prices_json -> dict : {'addy':'price * 10**30}

def price_format(x):
    return int(x)/10**30

def included_coins(x):
    return True if x in coins else False

def get_prices():
    response = urlopen(prices_url) # store the response of URL 
    prices_json = json.loads(response.read()) # storing the JSON response
    return prices_json

def time_price_data():
    
    p_json = get_prices()
    now = datetime.now()
    current_time = now.strftime("%I:%M:%S")
    print("Time:", current_time)
    
    for addy in p_json:
        if addy in coins:
            # print(coins[addy],price_format(p_json[addy]))
            if coins[addy] == 'WBTC/USD':
                wbtc_data.append([datetime.now().strftime("%m/%d/%Y"),current_time,'WBTC/USD',price_format(p_json[addy])])
            if coins[addy] == 'ETH/USD':
                eth_data.append([datetime.now().strftime("%m/%d/%Y"),current_time,'ETH/USD',price_format(p_json[addy])])
    return

In [47]:
green_light = True
while green_light:
    time_price_data()
    time.sleep(0.5)

Time: 09:30:47
Time: 09:30:48
Time: 09:30:49
Time: 09:30:50
Time: 09:30:51
Time: 09:30:53
Time: 09:30:54
Time: 09:30:55
Time: 09:30:56
Time: 09:30:57
Time: 09:30:58
Time: 09:31:00
Time: 09:31:01
Time: 09:31:02
Time: 09:31:04
Time: 09:31:05
Time: 09:31:06
Time: 09:31:08
Time: 09:31:09
Time: 09:31:10
Time: 09:31:11
Time: 09:31:13
Time: 09:31:14
Time: 09:31:15
Time: 09:31:16
Time: 09:31:17
Time: 09:31:18
Time: 09:31:19
Time: 09:31:20
Time: 09:31:21
Time: 09:31:22
Time: 09:31:24
Time: 09:31:25
Time: 09:31:27
Time: 09:31:28
Time: 09:31:28
Time: 09:31:29
Time: 09:31:31
Time: 09:31:32
Time: 09:31:33
Time: 09:31:34
Time: 09:31:35
Time: 09:31:36
Time: 09:31:37
Time: 09:31:38
Time: 09:31:38
Time: 09:31:40
Time: 09:31:41
Time: 09:31:43
Time: 09:31:44
Time: 09:31:45
Time: 09:31:47
Time: 09:31:48
Time: 09:31:49
Time: 09:31:50
Time: 09:31:51
Time: 09:31:52
Time: 09:31:53
Time: 09:31:54
Time: 09:31:54
Time: 09:31:56
Time: 09:31:57
Time: 09:31:58
Time: 09:31:59
Time: 09:32:00
Time: 09:32:01
Time: 09:3

KeyboardInterrupt: 

In [48]:

col_names = [['Date','Time','Pair','Price']]

#Formatting data and creating an CSV file and dataframe
with open('data.txt','w') as f:
    write = csv.writer(f)
    write.writerows(col_names)
    write.writerows(wbtc_data)

read_file = pd.read_csv('data.txt')
read_file.to_csv('data.csv', index=None)
df = pd.DataFrame(pd.read_csv('data.txt'))

f.close()

#rate(in seconds) is the time-scaling interval of VIEWING the data
def plot_rate(rate): 
    open = 0
    opens = {}
    closes = {}
    highs = {}
    lows = {}
    average = 0
    range_values = []

    dt = rate  #time interval(1 for 1s, 60 for 1m, 300 for 5m)

    # prices = df.loc[:,'Price']
    for i in range(df.shape[0]):
    # for i in range(106):
        t = df.iloc[i,1] #time
        if i+dt <= df.shape[0]: #required ammo for new candle
            dp = df.iloc[i,-1] #new price

            # if i!=0:
            range_values.append(dp)
            #Open and Close values
            if i % dt == 0: #new candle
                open = dp
                opens[t] = open
                if i != 0: #close is last value of interval
                    close = df.iloc[i+dt-1,-1]
                    closes[t] = close
                    lows[t] = np.min(range_values) #---low at end of interval t, NOT WHEN low occurs, important distinction 
                    highs[t] = np.max(range_values)
                    # print('Min/Max:',np.min(range_values),np.max(range_values))
                    # print(i,t,range_values)
                    # print('len:',len(range_values))
                    range_values = [] #reset for next interval
        else:  
            #---Since open is first, there'll be 1 more value in open than close/low/high, so we'll pad close with current open(0 difference)
            gen_time = str((datetime.strptime(list(opens.keys())[-1],'%I:%M:%S') + timedelta(seconds=dt)).time())
            closes[gen_time] = opens[list(opens.keys())[-1]]
            lows[gen_time] = lows[list(lows.keys())[-1]]
            highs[gen_time] = highs[list(highs.keys())[-1]]
            break

    open_data = list(opens.values())
    high_data = list(highs.values())
    low_data = list(lows.values())
    close_data = list(closes.values())
    times = list(opens.keys())


    fig = go.Figure(data=[go.Candlestick(x=times,
                        open=open_data, 
                        high=high_data,
                        low=low_data, 
                        close=close_data,name='WBTC')])

    fig.update_layout(
        title={
            'text' : 'WBTC',
            'y':0.9,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'})

    fig.show()

In [49]:
# plot_rate(1,'WBTC')
plot_rate(1)

In [50]:
plot_rate(60)#Check plot_rate_1 for output

In [6]:
plot_rate(5) #Check plot_rate_5 for output

In [147]:
plot_rate(60) #Check plot_rate_60 for output

In [148]:
plot_rate(300) #Check plot_rate_300 for output